In [ ]:
# import modules
from twitter import *
from urllib.error import HTTPError
import pandas as pd
import json
import time

In [ ]:
# temp util
def timestamp_attrs(data, col='created_at', tz='America/Bogota', passtz=True):
    '''
    '''
    if passtz:
        t = pd.to_datetime(
            data[col],
            utc=True,
            infer_datetime_format=True
        ).apply(lambda x: x.tz_convert(tz))
    else:
        t = pd.to_datetime(
            data[col],
            infer_datetime_format=True
        )

    data[f'{col}_timestamp'] = t
    data[f'{col}'] = t.dt.strftime('%Y-%m-%d')
    data[f'{col}_year'] = t.dt.year
    data[f'{col}_month'] = t.dt.month
    data[f'{col}_day'] = t.dt.day
    data[f'{col}_hour'] = t.dt.hour
    data[f'{col}_weekday'] = t.dt.dayofweek
    data[f'{col}_time_hour'] = t.dt.strftime('%H:%M:%S')
    
    return data

**`Twitter Auth`**

In [ ]:
# read credentials
with open('../_config/twitter_auth.json') as f:
    auth = json.load(f)
    f.close()

# Auth
Auth = OAuth(**auth)

# connection
cxn = Twitter(auth=Auth)

In [ ]:
# get followers
def get_followers(username, count=5000, get_all=True):
    '''
    '''
    followers = []
    params = {
        'screen_name': username,
        'cursor': -1,
        'stringify_ids': True,
        'count': count
    }
    
    users = cxn.followers.ids(**params)
    followers.extend(users['ids'])
    cursor = users['next_cursor']
    
    # iterate
    if get_all != True:
        return followers
    else:
        while cursor != 0:
            try:
                users = cxn.followers.ids(**params)
                followers.extend(users['ids'])
                cursor = users['next_cursor']
            except (TwitterHTTPError, HTTPError):
                status = cxn.application.rate_limit_status()
                
                remain = status['resources']['followers']['/followers/ids']['remaining']
                reset = status['resources']['followers']['/followers/ids']['reset']
                
                if remain <= 10:
                    sleep_for = int(reset - time.time()) + 10
                    time.sleep(sleep_for)
                
                continue
        
        return followers

In [ ]:
username = 'TJ16762413'
followers = get_followers(username)

In [ ]:
print (f'Number of followers {len(followers)}')

*`Explore followers`*

In [ ]:
division = 100
ids = [followers[i: i + division] for i in range(0, len(followers), division)]
all_ = []
for accounts in ids:
    users_lookup = cxn.users.lookup(user_id=','.join(accounts))
    all_.extend(users_lookup)
    
print (f'Metadata -> {len(all_)}')

In [ ]:
df = pd.DataFrame.from_dict(all_)
df.shape

*`load columns order`*

In [ ]:
order = [i.rstrip() for i in open('../assets/get_followers/columns_order.txt', encoding='utf-8')]

*`create dataset`*

In [ ]:
dataset = timestamp_attrs(df, tz='America/New_York')

In [ ]:
dataset = dataset[order].copy()

In [ ]:
dataset['created_at_timestamp'] = dataset['created_at_timestamp'].astype(str)

In [ ]:
path = f'../_data/followers_resilience/{username}_followers.xlsx'
dataset.to_excel(path, index=False)

In [ ]:
df[['screen_name', 'created_at']]